In [8]:
import os
import json
import datetime
import pandas as pd

In [9]:
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By

In [10]:
# load names downloaded from: 
# https://family.disney.com/articles/1000-most-popular-girl-names/ 
# https://family.disney.com/articles/1000-most-popular-boy-names/
with open('names.txt') as f:
    names = f.readlines()
names = [n.replace('\n','') for n in names]
df = pd.DataFrame(names)
df.columns = ['name']
names = df['name'].unique().tolist()

In [11]:
if not os.path.exists('results'):
    os.makedirs('results')

In [ ]:
# init browser
browser = Chrome(executable_path='/usr/local/bin/chromedriver')

run = True
while run:
    
    # get updated list of names done
    names_done = os.listdir('results')
    names_done = [n.replace('.json','') for n in names_done if n != '.DS_Store']
    
    # check if name already done
    name = ''
    for i, name in enumerate(names):
        if name in names_done:
            pass
        else:
            break
            
    if len(names) == len(names_done):
        break
            
    print(f"Begin checking {name}")
            
    if not os.path.exists('results/' + name + '.json'):

        print(f'{i+1}/{len(names)} Checking: {name}')

        # check if page already loaded
        loaded = False
        try:
            el = browser.find_element(By.XPATH, '/html/body/div[1]/div/main/div/div[1]/h2')
            if el.text.split(".eth")[0] == name.lower():
                loaded = True
        except:
            pass
        if not loaded:
            browser.get('https://app.ens.domains/name/' + name + '.eth')
            time.sleep(30)
        
        
        # check if we're getting rate limited
        ratelimited = False
        try:
            el = browser.find_element(By.XPATH, '/html/body/div[1]/div/div/h2')
            print('RATE LIMITED ', el.text)
            ratelimited = True
            time.sleep(30)
            browser.get('https://app.ens.domains/name/' + name + '.eth')
            time.sleep(15)
        except:
            pass
        
        
        # check if domain is registered
        try:            
            # get data
            els = browser.find_elements_by_class_name("em9ajyt7");
            registrant = els[0].text
            expiry = els[2].text

            result = {}
            result['name'] = name
            result['registered'] = 1
            result['expiry'] = expiry
            result['registrant'] = registrant

            # save result
            with open('results/' + name + '.json', 'w') as f:
                json.dump(result, f)

            print(f'{name} registered, expiry {expiry}')

        except Exception as e:
            print(e)
            try:
                # check if seeing page to register the name
                if "Registering a name requires you to complete" in browser.page_source:
                    result = {}
                    result['name'] = name
                    result['registered'] = 0
                    result['expiry'] = ''
                    result['registrant'] = ''

                    # save result
                    with open('results/' + name + '.json', 'w') as f:
                        json.dump(result, f)

                    print(f'{name} available')
                else:
                    time.sleep(30)
                
            except:
                print("ERROR - Page not registered and not available - sleeping 30 seconds and trying again")
                time.sleep(30)
                pass

Begin checking Eli
54/1921 Checking: Eli
list index out of range
Eli available
Begin checking Jonathan
55/1921 Checking: Jonathan
Jonathan registered, expiry 2025.05.04 at 07:06 (UTC+02:00)
Begin checking Connor
56/1921 Checking: Connor
RATE LIMITED  invalid response - 429
